In [7]:
import datetime
import os
import time
from tqdm import tqdm
import torch
import h5py
import numpy as np
from torchinfo import summary
import math
from torch.autograd import Variable
import pickle

In [8]:
# custom module
import model, dataloader, train
import importlib
importlib.reload(model)
importlib.reload(dataloader)
importlib.reload(train)
from model import *
from dataloader import *
from train import train
from test import test

# Parameters

In [9]:
config = {
    "root" : "./data/pickles/",
    "train_feature" : "AtrialFibrilation_feature.pickle",
    "train_target" : "AtrialFibrilation_target.pickle",
    "test_feature" : "AtrialFibrilation_feature_test.pickle",
    "test_target" : "AtrialFibrilation_target_test.pickle",
    "labelencodedict" : { 'n' : 0,
                    's' : 1,
                    't' :2,
                   },
    "epochs" : 3000,
    "learning_rate" : 1e-4,
    # model 
    "first_out_channels" : 32,
    "first_kernel_size" : 5,
    "second_in_channels" : 32,
    "second_out_channels" : 16,
    "second_kernel_size" : 3,
    "dim_model" : 64,
    "dim_inner" : 128,
    "num_heads" : 8,
    "dropout_rate" : 0.15,
    "squeeze_factor" : 2,
    "sec_kernel_size" : 3,
    "sec_stride" : 1,
    "N" : 3,
    "gamma" : 2,
    "u" : 2,        
    "weight_decay" : 0.1,
       "device" : torch.device('cuda' if torch.cuda.is_available() else 'cpu')
}

# Main

In [11]:
if __name__ == "__main__":
    epochs = config["epochs"]
    learning_rate = config["learning_rate"]
    weight_decay = config["weight_decay"]

    X_train, y_train, train_y, X_test, y_test, test_y = load_dataset(config)
    
    config["num_samples"] = X_train.shape[0]
    config["first_in_channels"] = X_train.shape[1]
    config["L"] = X_train.shape[2]
    config["length"] = X_train.shape[2]
    
    md = Net(
                dim_model = config["dim_model"],
                gamma = config["gamma"],
                u = config["u"],
                ys = y_train,
                first_in_channels = config["first_in_channels"],
                first_out_channels = config["first_out_channels"],
                first_kernel_size = config["first_kernel_size"],
                second_in_channels = config["second_in_channels"],
                second_out_channels = config["second_out_channels"],
                second_kernel_size = config["second_kernel_size"],
                N = config["N"],  #slice
                L = config["L"], # slice
                dim_inner = config["dim_inner"],
                num_samples = config["num_samples"], 
                length = config["length"], 
                num_heads = config["num_heads"],
                dropout_rate = config["dropout_rate"],
                squeeze_factor = config["squeeze_factor"],
                sec_kernel_size = config["sec_kernel_size"],
                sec_stride = config["sec_stride"],
                num_classes = len(config["labelencodedict"].keys())).to(config['device'])
    optimizer = torch.optim.Adam(md.parameters(), weight_decay=weight_decay, lr=learning_rate)
    criterion = torch.nn.CrossEntropyLoss()

    train_loss, train_acc = train(X_train, train_y, optimizer, model = md, epochs = epochs)
    md.ys = y_test
    test_loss, test_acc = test(X_test, test_y, model = md)

C:\Users\yuins\Hanyang_23_1st\AI\model.py:39: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  attention_prob = torch.nn.functional.softmax(scaled_att_score)
C:\Users\yuins\Hanyang_23_1st\AI\model.py:241: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  attention_prob = torch.nn.functional.softmax(scaled_att_score)


Epoch: 0, Loss: 0.7874, Train acc: 95.5556%
Epoch: 1, Loss: 0.8008, Train acc: 82.2222%
Epoch: 2, Loss: 0.6747, Train acc: 100.0000%
Epoch: 3, Loss: 0.6599, Train acc: 100.0000%
Epoch: 4, Loss: 0.6687, Train acc: 100.0000%
Epoch: 5, Loss: 0.6334, Train acc: 100.0000%
Epoch: 6, Loss: 0.6175, Train acc: 100.0000%
Epoch: 7, Loss: 0.6097, Train acc: 95.5556%
Epoch: 8, Loss: 0.5730, Train acc: 100.0000%
Epoch: 9, Loss: 0.5570, Train acc: 100.0000%
Epoch: 10, Loss: 0.5245, Train acc: 100.0000%
Epoch: 11, Loss: 0.3480, Train acc: 100.0000%
Epoch: 12, Loss: 0.3773, Train acc: 100.0000%
Epoch: 13, Loss: 0.3129, Train acc: 95.5556%
Epoch: 14, Loss: 0.3785, Train acc: 95.5556%
Epoch: 15, Loss: 0.2830, Train acc: 100.0000%
Epoch: 16, Loss: 0.2662, Train acc: 100.0000%
Epoch: 17, Loss: 0.2340, Train acc: 100.0000%
Epoch: 18, Loss: 0.2881, Train acc: 100.0000%
Epoch: 19, Loss: 0.1963, Train acc: 100.0000%
Epoch: 20, Loss: 0.1536, Train acc: 100.0000%
Epoch: 21, Loss: 0.1867, Train acc: 100.0000%
Epo

KeyboardInterrupt: 

In [ ]:
md.ys = y_test
test_loss, test_acc = test(X_test, test_y, model = md)